In [1]:
import torch
import json
from PIL import Image


import torchvision
import numpy as np
from torchvision import models , transforms

import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"
import matplotlib
matplotlib.use('TkAgg')  # 使用 Tkinter 支持的后端

In [2]:
#版本信息
print(f'pytorch version {torch.__version__}') ; print(f'torchvision version {torchvision.__version__}')

pytorch version 1.12.1+cu113
torchvision version 0.13.1+cu113


In [3]:
#调用GPU
def try_gpu(i=0):
    if torch.cuda.device_count() > i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [4]:
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)
net.eval()

D:\anaconda3\envs\douhaunmin\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
D:\anaconda3\envs\douhaunmin\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
# 对图片进行预处理
class BaseTransform():
    def __init__(self , resize , mean , std):
        self.basetransforms = transforms.Compose([
            transforms.Resize(resize) , # 较短的边为resize的大小 ， 较长边根据宽高比调整
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean , std)
        ])
    def __call__(self , img):
        return self.basetransforms(img)

In [5]:
img_path = "../Downloads/pytorch_advanced-master/1_image_classification/data/goldenretriever-3724972_640.jpg"
ILSVRC_class_index = json.load(open("../Downloads/pytorch_advanced-master/1_image_classification/data/imagenet_class_index.json" , "r"))

In [7]:
img = Image.open(img_path) ; plt.imshow(img) ; plt.show()

In [8]:
#图像处理
resize , mean , std = 224 , (0.485 , 0.456 , 0.406) , (0.229 , 0.224 , 0.225)
transform = BaseTransform(resize , mean , std)
img_transform = transform(img)

In [9]:
img_transformed = img_transform.numpy().transpose((1 , 2 , 0))
img_transformed = np.clip(img_transformed , 0 , 1)
plt.imshow(img_transformed) ; plt.show()

In [10]:
# 预测
class ILSVRCPredictor():
    def __init__(self,  class_index):
        self.class_index = class_index
    def predict_max(self , out):
        maxid = np.argmax(out.detach().numpy())
        predict_label_name = self.class_index[str(maxid)][1]
        return predict_label_name

In [11]:
predictor = ILSVRCPredictor(ILSVRC_class_index)
inputs = img_transform.unsqueeze_(0)

In [12]:
out = net(inputs)
result = predictor.predict_max(out)
print(result)

golden_retriever
